In [1]:
# loading modules
import ee
import geemap

# initialize ee 
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()



In [2]:
# CASE 1: Pakistan Floods 2022
# inputs
# Define the Area of Interest (aoi)

name = 'Pakistan_Floods_2022'
aoi = ee.Geometry.Polygon(
    [[[67.98392410136336, 26.049909335428502],
      [67.98392410136336, 25.42892423506662],
      [68.59778518534773, 25.42892423506662],
      [68.59778518534773, 26.049909335428502]]])

startDate = ee.Date('2022-03-01')
endDate = ee.Date('2022-08-20')
predays = 60
postdays = 20

# optional
zvv_value = -3
zvh_val = -3
water_value = 75
elev_value = 800
slope_value = 10
num_samples = 1000

In [4]:
a = ee.FeatureCollection("projects/temporaryproj/assets/fs_cities_boundary_combined")
# print all names of features with property name
a.aggregate_array('name').getInfo()

['Bangkok', 'Jakarta', 'Manila', 'Hochiminh', 'Mumbai']

In [ ]:
# Cities Info
name = 'Mumbai'
startDate = ee.Date('2017-03-01')
endDate = ee.Date('2017-08-22')
predays = 30
postdays = 10


name = 'Bangkok'
startDate = ee.Date('2023-01-01')
endDate = ee.Date('2023-09-25')
predays = 60
postdays = 10

In [3]:
# CASE 2: Mumbai
# inputs
# Define the Area of Interest (aoi)
name = 'Bangkok'
startDate = ee.Date('2023-01-01')
endDate = ee.Date('2023-09-25')
predays = 60
postdays = 10

fsm_cities_box = ee.FeatureCollection("projects/temporaryproj/assets/fs_cities_boundarybox_combined")
aoi = fsm_cities_box.filter(ee.Filter.eq('name', f'{name}_aoi')).geometry()
fsm_cities = ee.FeatureCollection("projects/temporaryproj/assets/fs_cities_boundary_combined")
aoi_city = fsm_cities.filter(ee.Filter.eq('name', name)).geometry()

zvv_value = -3
zvh_val = -3
water_value = 75
elev_value = 800
slope_value = 10


In [ ]:
def get_s1_col(date, days):
  filters = [
    ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
    ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
    ee.Filter.Or(ee.Filter.equals("instrumentMode", "IW"),ee.Filter.equals("instrumentMode", "SM")),
    ee.Filter.bounds(aoi),
    ee.Filter.eq('resolution_meters', 10),
    ee.Filter.date(date, date.advance(days+1, 'day'))
  ]
  
  s1_col = ee.ImageCollection('COPERNICUS/S1_GRD').filter(filters)

  return s1_col


s1_pre = get_s1_col(startDate, predays).select(['VV', 'VH'])
s1_post = get_s1_col(endDate, postdays).select(['VV', 'VH'])

# images in s1 pre and post
print('Images in S1 Pre: ',s1_pre.size().getInfo())
print('Images in S1 Post: ',s1_post.size().getInfo())

asc = ee.Filter.eq("orbitProperties_pass", "ASCENDING")
des = ee.Filter.eq("orbitProperties_pass", "DESCENDING")

# Function to calculate z-score
def calc_zscore(s1_pre, s1_post, direction):
    base_mean = s1_pre.filter(ee.Filter.equals('orbitProperties_pass', direction)).mean()

    anom = s1_post.filter(ee.Filter.equals('orbitProperties_pass', direction)) \
        .mean().subtract(base_mean).set({'system:time_start': s1_post.get('system:time_start')})

    base_sd = s1_pre.filter(ee.Filter.equals('orbitProperties_pass', direction)) \
        .reduce(ee.Reducer.stdDev()).rename(['VV', 'VH'])

    return anom.divide(base_sd).set({'system:time_start': anom.get('system:time_start')})

def zscore_combined():
  zscore_des = calc_zscore(s1_pre, s1_post,'DESCENDING')
  zscore_asc = calc_zscore(s1_pre, s1_post,'ASCENDING')

  # Calculate mean of z-scores for DESCENDING and ASCENDING orbits
  zscore = ee.ImageCollection.fromImages([zscore_des, zscore_asc]).mean().clip(aoi)

  return zscore

# caculate zscore for only ascending orbits
def zscore_asc():
  base_mean = s1_pre.filter(asc).mean()
  anom = s1_post.filter(asc)\
                .mean().subtract(base_mean)
  base_sd = s1_pre.filter(asc)\
                .reduce(ee.Reducer.stdDev()).rename(['VV', 'VH'])
  zscore = anom.divide(base_sd)
  return zscore

# caculate zscore for only descending orbits
def zscore_des():
  base_mean = s1_pre.filter(des).mean()
  anom = s1_post.filter(des)\
                .mean().subtract(base_mean)
  base_sd = s1_pre.filter(des)\
                .reduce(ee.Reducer.stdDev()).rename(['VV', 'VH'])
  zscore = anom.divide(base_sd)
  return zscore

def zscore_cond():
  cond_a = s1_pre.filter(ee.Filter.eq("orbitProperties_pass", "ASCENDING")).size().gt(0)
  cond_b = s1_pre.filter(ee.Filter.eq("orbitProperties_pass", "DESCENDING")).size().gt(0)
  cond_c = s1_post.filter(ee.Filter.eq("orbitProperties_pass", "ASCENDING")).size().gt(0)
  cond_d = s1_post.filter(ee.Filter.eq("orbitProperties_pass", "DESCENDING")).size().gt(0)
  
  cond_asc = cond_a.And(cond_c)
  cond_des = cond_b.And(cond_d)
  if cond_asc == 1:
    images = zscore_asc()
    zscore = ee.ImageCollection.fromImages([images]).mean().clip(aoi)
  else:
    images = zscore_des()
    zscore = ee.ImageCollection.fromImages([images]).mean().clip(aoi)
  
  return zscore

# Check if both ascending and descending collections are empty
pre_hasAscending = s1_pre.filter([asc]).size().gt(0)
pre_hasDescending = s1_pre.filter([des]).size().gt(0)
post_hasAscending = s1_post.filter([asc]).size().gt(0)
post_hasDescending = s1_post.filter([des]).size().gt(0)

cond = pre_hasAscending.And(pre_hasDescending).And(post_hasAscending).And(post_hasDescending)

if cond == 1:
  zscore = zscore_combined()
else:
  zscore = zscore_cond()


# Flood mask function
def mapFloods(z, zvv_thd=zvv_value, zvh_thd=zvh_val, pow_thd=water_value, elev_thd=elev_value, slp_thd=slope_value):

    # JRC water mask
    jrc = ee.ImageCollection("JRC/GSW1_1/MonthlyHistory").filterDate('2016-01-01', '2022-01-01')
    jrcvalid = jrc.map(lambda x: x.gt(0)).sum()
    jrcwat = jrc.map(lambda x: x.eq(2)).sum().divide(jrcvalid).multiply(100)
    jrcmask = jrcvalid.gt(0)
    ow = jrcwat.gte(ee.Image(pow_thd))

    # Add elevation and slope masking using fabdem
    elevation = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").mosaic() \
        .setDefaultProjection('EPSG:3857', None, 30)
    slope = ee.Terrain.slope(elevation)

    # Classify floods
    vvflag = z.select('VV').lte(ee.Image(zvv_thd))
    vhflag = z.select('VH').lte(ee.Image(zvh_thd))

    flood_class = ee.Image(0).add(vvflag) \
        .add(vhflag.multiply(2)) \
        .where(ow.eq(1), 4) \
        .rename('flood_class') \
        .where(elevation.gt(elev_thd).multiply(ow.neq(1)), 0) \
        .where(slope.gt(slp_thd).multiply(ow.neq(1)), 0)
    # add class 1,2,3 from flood_class and create a new layer with single band
    flood_layer = flood_class.where(flood_class.eq(1), 1) \
        .where(flood_class.eq(2), 1) \
        .where(flood_class.eq(3), 1) \
        .where(flood_class.eq(4), 2) \
        .where(jrcmask.eq(0), 2) \
        .where(flood_class.eq(0), 2) \
        .selfMask() \
        .rename('label')
    return flood_class.clip(aoi), flood_layer.clip(aoi)

# Mask z-score threshold
flood_class, flood_layer = mapFloods(zscore)

# sampling


# Create the 'sample' feature collection


sample = flood_layer.stratifiedSample(
    numPoints=num_samples,
    classBand='label',
    region=aoi,
    scale=10,
    seed=5,
    tileScale=1.5,
    geometries=True
)

# initially non flood is labeled 2, we will now change 2 to 0
def updateFeature(feature):
    value = feature.get('label')
    cond = ee.Algorithms.IsEqual(value, ee.Number(2))
    updated_value = ee.Algorithms.If(cond, ee.Number(0), value)
    feature = feature.set('label', updated_value)
    return feature
label = sample.map(updateFeature)



# inputs in classificaation

image = s1_post.mean().clip(aoi)
bands = ['VV', 'VH']


# training
sample_all = image.select(bands).sampleRegions(**{
    'collection': label,
    'properties': ['label'],
    'scale': 10
})

sample_all = sample_all.randomColumn()

split = 0.9
training = sample_all.filter(ee.Filter.lt('random', split))
validation = sample_all.filter(ee.Filter.gte('random', split))


classifier = ee.Classifier.smileRandomForest(115).train(**{
    'features': training,
    'classProperty': 'label',
    'inputProperties': bands
})
results = image.select(bands).classify(classifier)

sample_sus = results.stratifiedSample(
    numPoints=num_samples,
    classBand='label',
    region=aoi,
    scale=10,
    seed=5,
    tileScale=1.5,
    geometries=True
)

# Susceptibility

In [ ]:
# Variables Preperation for Susceptibility


# varianles used DEM, Slope, Aspect, NDVI, NDWI, NDBI, NDSI

# Applies scaling factors.
year = endDate.get('year')
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    
    return image.addBands(opticalBands, None, True)\
              .addBands(thermalBands, None, True)

def remapper(image):
        remapped = image.remap([1,2,4,5,7,8,9,10,11],[1,2,3,4,5,6,7,8,9])
        return remapped

# datasets
dem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM")\
        .filterBounds(aoi)\
        .mosaic()\
        .clip(aoi)

slope = ee.Terrain.slope(dem)
aspect = ee.Terrain.aspect(dem)


l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')\
        .filterBounds(aoi)\
        .filter(ee.Filter.calendarRange(year, year, 'year'))\
        .map(applyScaleFactors)\
        .filter(ee.Filter.lt('CLOUD_COVER', 10))\
        .median()\
        .clip(aoi)


ndvi = l9.normalizedDifference(['SR_B5', 'SR_B4'])
ndwi = l9.normalizedDifference(['SR_B3', 'SR_B5'])
ndbi = l9.normalizedDifference(['SR_B6', 'SR_B5'])
ndsi = l9.normalizedDifference(['SR_B2', 'SR_B5'])

l9 = l9.select(['SR_B1','SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])

image_sus = l9.addBands(dem).addBands(slope).addBands(aspect).addBands(ndvi).addBands(ndwi).addBands(ndbi).addBands(ndsi).clip(aoi)

bands_sus = image_sus.bandNames().getInfo()

# training
sample_all_sus = image_sus.select(bands_sus).sampleRegions(**{
    'collection': label,
    'properties': ['label'],
    'scale': 30
})

sample_all_sus = sample_all_sus.randomColumn()

split = 0.9
training_sus = sample_all_sus.filter(ee.Filter.lt('random', split))
validation_sus = sample_all_sus.filter(ee.Filter.gte('random', split))


classifier_sus = ee.Classifier.smileRandomForest(115).train(**{
    'features': training_sus,
    'classProperty': 'label',
    'inputProperties': bands_sus
})

classifier_prob = classifier_sus.setOutputMode('PROBABILITY')
results_prob = image_sus.select(bands).classify(classifier_prob)

Map.addLayer(results_prob, {'min': 0, 'max': 1, 'palette': ['green', 'blue', 'red']}, 'Flood Susceptibility');


In [ ]:
Map.addLayer(results, {'min': 1, 'max': 3, 'palette': ['white', 'red', '233CF0']}, 'Flood Classified');
Map.addLayer(results_prob, {'min': 0, 'max': 1, 'palette': ['white', 'blue', '233CF0']}, 'Flood Susceptibility');

# map
Map = geemap.Map()
Map.centerObject(aoi)
Map.addLayer(aoi, {}, 'Area of Interest')
Map.addLayer(zscore.select('VV'), {'min': -7, 'max': 7, 'palette': ['red', 'white', 'blue']}, 'ZScore', False)
Map.addLayer(flood_class, {'min': 0, 'max': 4, 'palette': ['#E3E3E3','#FFB100', '#FFB100', '#3E9DFF','#031DC9']}, 'flood classes', False)
Map.addLayer(flood_layer, {'min': 1, 'max': 2, 'palette': ['blue', 'white']}, 'flood layer', False)
Map.addLayer(label, {}, 'Label')
Map

In [30]:
classifier_explain = classifier.explain()
variable_importance = ee.Feature(None, ee.Dictionary(classifier_explain).get('importance'))
#print('Classifier Explain: ', classifier_explain)
print('Variable Importance: ', variable_importance.getInfo())
train_accuracy = classifier.confusionMatrix()
print('Train Accuracy', train_accuracy.getInfo())
print('Overall Accuracy: ', train_accuracy.accuracy().getInfo())
print('Kappa: ', train_accuracy.kappa().getInfo())
print('Producer Accuracy: ', train_accuracy.producersAccuracy().getInfo())
print('Consumer Accuracy: ', train_accuracy.consumersAccuracy().getInfo())

validated = validation.classify(classifier)
print('Validation', validated.first().getInfo())
test_accuracy = validated.errorMatrix('label', 'classification')
print('Validation Accuracy', test_accuracy.getInfo())
print('Validation Overall Accuracy: ', test_accuracy.accuracy().getInfo())
print('Validation Kappa: ', test_accuracy.kappa().getInfo())
print('Validation Producer Accuracy: ', test_accuracy.producersAccuracy().getInfo())
print('Validation Consumer Accuracy: ', test_accuracy.consumersAccuracy().getInfo())

f1_train = train_accuracy.fscore().getInfo()[1]
print('F1 Score Train: ', f1_train)

f1_test = test_accuracy.fscore().getInfo()[1]
print('F1 Score Test: ', f1_test)



Variable Importance:  {'type': 'Feature', 'geometry': None, 'properties': {'VH': 2103.865866514957, 'VV': 2078.6021142779514}}
Train Accuracy [[687, 8], [6, 678]]
Overall Accuracy:  0.9898477157360406
Kappa:  0.9796946092795193
Producer Accuracy:  [[0.9884892086330935], [0.9912280701754386]]
Consumer Accuracy:  [[0.9913419913419913, 0.9883381924198251]]
Validation {'type': 'Feature', 'geometry': None, 'id': '1_0', 'properties': {'VH': -18.27881045056565, 'VV': -8.568157684616803, 'classification': 1, 'label': 1, 'random': 0.7433594988094623}}
Validation Accuracy [[200, 105], [119, 197]]
Validation Overall Accuracy:  0.6392914653784219
Validation Kappa:  0.27893258135749593
Validation Producer Accuracy:  [[0.6557377049180327], [0.6234177215189873]]
Validation Consumer Accuracy:  [[0.6269592476489029, 0.652317880794702]]


In [4]:
# exporting
geemap.ee_to_shp(label, filename=f'../data/{name}_label.shp')
#geemap.ee_export_image_to_drive(flood_layer, description='flood_layer', folder='s1_post', region=aoi, scale=10)

Generating URL ...
Please wait ...
Data downloaded to e:\current_proj\FMS\codes\data\Mumbai_label.shp
